In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
#A1, B3 for products55-81


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
demand = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\historical_demand.csv")
forecast = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\sample_submission\\demand_forecast.csv")
delivery_cost = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\delivery_cost.csv")
production_cost = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\production_cost.csv")
demand_price = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\demand_price.csv")
production_capacity = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\production_capacity.csv")
changeover_days = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\train\\changeover_days.csv")
manufacture_sequence = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\sample_submission\\manufacture_sequence.csv")
shipping_region = pd.read_csv("C:\\Users\\kkvamshee\\Desktop\\ML\\Mckinsey\\sample_submission\\shipping_region.csv")

In [3]:
count=0
for product in demand['Product_ID'].unique()[:60]:
    for region in demand['Region'].unique():
        temp = demand[(demand['Product_ID']==product)&(demand['Region']==region)]
        if not(temp['Demand'].sum() == 0):
            count+=1
count

118

In [64]:
train = demand[demand['Month']<33]
valid = demand[demand['Month']>=33]

In [67]:
for i in demand.columns:
    if type(train[i][0])==str:
        le = LabelEncoder()
        train[i] = le.fit_transform(train[i])
        valid[i] = le.transform(valid[i])

C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [70]:
tree = DecisionTreeRegressor(max_depth=3, criterion='mse')
tree.fit(train.drop('Demand', axis=1), train['Demand'])

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [82]:
pred=[]
for product in demand['Product_ID'].unique():
    for region in demand['Region'].unique():
        temp = demand[(demand['Product_ID']==product)&(demand['Region']==region)]
        avg = temp['Demand'].mean()
        for i in range(3):
            pred.append(avg)

In [85]:
forecast['Demand'] = pred

In [92]:
forecast.to_csv('demand_forecast.csv', index=False)

In [45]:
demand['Region'].unique()

array(['R1', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18',
       'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9'], dtype=object)

In [4]:
month=37
monthly_demand = pd.Series()
for product in forecast['Product_ID'].unique():
    prod_demand = forecast[(forecast['Month']==37)&(forecast['Product_ID']==product)]['Demand'].sum()
    monthly_demand[product] = prod_demand

In [17]:
mforecast = forecast[forecast['Month']==37]

In [18]:
monthly_demand

P1        0.000000
P10       0.000000
P11       0.000000
P12       0.000000
P13     275.068611
P14     271.125000
P15     526.563889
P16     147.830000
P17      46.951667
P18       0.000000
P19     132.142500
P2        0.000000
P20       0.000000
P21       0.000000
P22     612.257778
P23    1380.254444
P24     458.715556
P25      95.571667
P26     211.361111
P27      30.546389
P28       0.000000
P29      70.998333
P3      401.106944
P30       0.000000
P31     250.951667
P32       0.000000
P33     149.732500
P34     208.625000
P35    1268.023333
P36      64.066944
          ...     
P56       0.000000
P57      90.498611
P58     221.559444
P59       0.000000
P6        0.000000
P60     100.697222
P61      64.231667
P62     126.701944
P63       0.000000
P64     108.526944
P65      45.509722
P66     165.241389
P67      88.023889
P68     253.691944
P69     102.761667
P7        0.000000
P70       0.000000
P71     131.469722
P72     331.385833
P73     253.917222
P74      75.455833
P75     100.

In [7]:
production_places = ['A' for i in range(2*18*81)] + ['B' for i in range(3*18*81)] + ['C' for i in range(18*81)]

In [8]:
production_lines = []
for i in [1, 2, 1, 2, 3, 1]:
    production_lines += [i for j in range(18*81)]

In [9]:
Regions=[]
for j in range(6):
    for region in demand['Region'].unique():
        Regions += [region for i in range(81)]

In [10]:
Products=[]
for a in range(6):
    for b in range(18):
        for product in demand['Product_ID'].unique():
            Products.append(product)

In [11]:
df = pd.DataFrame({'Plant':production_places,
                  'Line':production_lines,
                  'Region':Regions,
                  'Product_ID':Products})

In [12]:
def pred_production_cost(plant, product):
    plant_map = {'A':0, 'B':1, 'C':2}
    idx = int(product[1:]) + plant_map[plant]*81 - 1
    return production_cost.loc[idx]['Production_cost']

cost=[]
for plant, product in zip(df['Plant'], df['Product_ID']):
    cost.append(pred_production_cost(plant, product))
    
df['Production_cost'] = cost

In [13]:
plant_map = {'A':0, 'B':1, 'C':2}
def pred_delivery_cost(plant, region):
    idx = int(region[1:]) + plant_map[plant]*18 - 1
    return delivery_cost.loc[idx]['Delivery_cost']

cost=[]
for plant, region in zip(df['Plant'], df['Region']):
    cost.append(pred_delivery_cost(plant, region))
    
df['Delivery_cost'] = cost

In [14]:
def pred_demand_price(product, region):
    idx = (int(region[1:])-1)*81 + int(product[1:]) - 1
    return demand_price.loc[idx]['Demand_price']

cost=[]
for product, region in zip(df['Product_ID'], df['Region']):
    cost.append(pred_demand_price(product, region))
    
df['Demand_price'] = cost

In [19]:
def product_transform(product):
    if len(product)==2:
        return 'P0'+product[-1]
    else:
        return product
    
def region_transform(product):
    if product[0]=='P':
        return 'R'+product[1:]
    else:
        return product

mforecast['Product_ID'] = mforecast['Product_ID'].map(product_transform)
mforecast['Region'] = mforecast['Region'].map(region_transform)

C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
mforecast.sort_values(by=['Region', 'Product_ID'], inplace=True)
mforecast.index = np.arange(mforecast.shape[0])

C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
def pred_demand(product, region):
    idx = (int(region[1:])-1)*81 + int(product[1:]) - 1
    return mforecast.loc[idx]['Demand']

cost=[]
for product, region in zip(df['Product_ID'], df['Region']):
    cost.append(pred_demand(product, region))
    
df['Demand'] = cost

In [22]:
df['Revenue'] = df['Demand_price'] - df['Delivery_cost'] - df['Production_cost']

In [23]:
plant_line_map = {('A', 1):0,
                 ('A', 2):1,
                 ('B', 1):2,
                 ('B', 2):3,
                 ('B', 3):4,
                 ('C', 1):5,}
plant_line_inv = {v: k for k, v in plant_line_map.items()}

In [24]:
def pred_capacity(line, plant, product):
    idx = plant_line_map[(plant, line)]*81 + int(product[1:]) - 1
    return production_capacity.loc[idx]['Capacity']
capacity = []
for line, plant, product in zip(df['Line'], df['Plant'], df['Product_ID']):
    capacity.append(pred_capacity(line, plant, product))
df['Capacity'] = capacity

In [25]:
plant_line=[]
for plant, line in zip(df['Plant'], df['Line']):
    plant_line.append((plant, line))
df['plant_line'] = plant_line

In [26]:
df1 = df[(df['Demand']!=0)&(df['Demand']>0)&(df['Capacity']!=0)]

In [27]:
df[(df['Production_cost']==0)&(df['Capacity']!=0)]['Demand']

7317     0.000000
7398    46.995000
7479     0.000000
7560     0.000000
7641     0.000000
7722     0.000000
7803     0.000000
7884    39.755833
7965    51.426389
8046     3.975278
8127     0.000000
8208    18.844722
8289     6.346944
8370     0.000000
8451     0.000000
8532    41.280833
8613     0.000000
8694     0.000000
Name: Demand, dtype: float64

In [28]:
quarter_demand = monthly_demand*3
month1_demand = monthly_demand

In [29]:
outliers = df[(df['Revenue']<0)&(df['Production_cost']!=0)&(df['Demand']!=0)]['Product_ID'].unique()

In [30]:
outliers

array(['P19', 'P3', 'P33', 'P34', 'P43', 'P47', 'P51', 'P57', 'P60', 'P61',
       'P64', 'P67', 'P68', 'P72', 'P74', 'P75', 'P77', 'P22', 'P16',
       'P36', 'P13', 'P65', 'P66', 'P58', 'P71', 'P8', 'P9', 'P62', 'P69',
       'P73', 'P35', 'P26', 'P39', 'P31', 'P25', 'P37', 'P15', 'P23', 'P4',
       'P27', 'P29', 'P14', 'P17', 'P24', 'P40', 'P41'], dtype=object)

In [31]:
month1 = np.empty((6, 30))
month3 = np.empty((6, 30))
month2 = np.empty((6, 30))

In [32]:
def ceil(num):
    if int(num)==num:
        return num
    else:
        return int(num)+1

In [33]:
df1['Profit'] = df1['Demand']*df1['Revenue']

C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
temp = []
for product in df1['Product_ID']:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
df1['Product_ID'] = temp

C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [35]:
temp = []
for product in monthly_demand.index:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
monthly_demand.index = temp

In [38]:
cd = changeover_days.copy()

In [39]:
temp = []
for product in cd['From']:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
cd['From'] = temp

temp = []
for product in cd['To']:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
cd['To'] = temp

In [40]:
def check_line(lines, line):
    res = 0
    for i in lines:
        if i==line:
            res+=1
    return bool(res)

def check(lines):
    temp = []
    if check_line(lines, ('B', 1)):
        temp.append(('B', 1))
    
    if check_line(lines, ('A', 2)):
        temp.append(('A', 2))
        
    if check_line(lines, ('B', 2)):
        temp.append(('B', 2))

    for i in lines:
        if not check_line(temp, i):
            temp.append(i)
    return temp

In [920]:
%time
count=0
line_demand=[0 for i in range(6)]
produced_quantity={}
month1 = np.chararray((6, 30), itemsize=3)
vacancy = {('A', 1):0,
            ('A', 2):0,
            ('B', 1):0,
            ('B', 2):0,
            ('B', 3):0,
            ('C', 1):0}

last_product = {('A', 1):0,
            ('A', 2):0,
            ('B', 1):0,
            ('B', 2):0,
            ('B', 3):0,
            ('C', 1):0}

for product in sorted(df1['Product_ID'].unique()):
    
    temp = df1[df1['Product_ID']==product]
    temp.sort_values(by='Profit', inplace=True, ascending=False)
    
    lines = check(temp['plant_line'].unique())
    
    flag=1
        
    while(flag):
        e=1
        try:
            line = lines[flag-1]
        except:
            flag=0
            e=0
            print('e', product, end=' ')
            #pass
        line_id = plant_line_map[line]
        if last_product[line] == 0:
            last_product[line] = product
        change = cd[(cd['From']==last_product[line])&(cd['To']==product)]['Days'].values[0]
        cap = temp[temp['plant_line']==line]['Capacity'].values[0]
        dem = monthly_demand[product]
        dur = ceil(dem/(2*cap))
        
        if e==0:
            print(dur , change)
            #pass
        
        if vacancy[line]+dur+change < 30:
            flag=0
            month1[line_id, vacancy[line]+change:vacancy[line]+dur+change] = product
            #print(product, vacancy[line], vacancy[line]+dur-1, vacancy[line]+dur+change, line)
            vacancy[line] += dur+change
            last_product[line] = product
        else:
            flag+=e
    count+=e
    if e:
        print(count)
        #pass

Wall time: 0 ns
253.917222222 56 3
1


C:\Users\kkvamshee\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [695]:
vacancy

{('A', 1): 29,
 ('A', 2): 25,
 ('B', 1): 28,
 ('B', 2): 10,
 ('B', 3): 29,
 ('C', 1): 20}

In [696]:
month1

chararray([[b'P57', '', '', b'P58', b'P58', '', '', b'P64', '', '', b'P65', '',
        '', b'P66', b'P66', '', '', b'P67', '', '', b'P68', b'P68', b'P68',
        '', '', b'P69', '', '', b'P74', ''],
       [b'P03', '', '', b'P13', '', b'P22', b'P22', b'P22', '', b'P33', '',
        b'P34', '', b'P36', '', '', '', b'P43', b'P43', b'P43', '', b'P47',
        '', b'P51', b'P51', '', '', '', '', ''],
       [b'P04', b'P04', b'P04', b'P04', b'P04', b'P04', b'P04', b'P04',
        b'P04', b'P04', b'P04', b'P04', '', b'P08', b'P08', '', b'P09',
        b'P09', b'P09', b'P09', '', '', b'P15', b'P15', '', b'P19', '',
        b'P26', '', ''],
       [b'P35', b'P35', b'P35', b'P35', b'P35', '', b'P37', b'P37', '',
        b'P39', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', ''],
       [b'P23', b'P23', b'P23', b'P23', b'P23', '', b'P25', '', b'P31', '',
        '', '', '', '', '', '', '', '', '', b'P60', '', '', b'P61', '', '',
        b'P62', '', '', b'P7

In [697]:
#p4 to b2
month1[3][-13:] = 'P04'
month1[2][:12] = ''
#p23 to b1
#removing P25,P31
month1[2][:5] = 'P23'
month1[4][:10] = ''
#adding 72,73,75,77 in b3
month1[4][:2] = 'P72'
month1[4][4:6] = 'P73'
month1[4][8] = 'P75'
month1[4][11:14] = 'P77'

In [707]:
month1

chararray([[b'P57', '', '', b'P58', b'P58', '', '', b'P64', '', '', b'P65', '',
        '', b'P66', b'P66', '', '', b'P67', '', '', b'P68', b'P68', b'P68',
        '', '', b'P69', '', '', b'P74', ''],
       [b'P03', '', '', b'P13', '', b'P22', b'P22', b'P22', '', b'P33', '',
        b'P34', '', b'P36', '', '', '', b'P43', b'P43', b'P43', '', b'P47',
        '', b'P51', b'P51', '', '', '', '', ''],
       [b'P23', b'P23', b'P23', b'P23', b'P23', '', '', '', '', '', '', '',
        '', b'P08', b'P08', '', b'P09', b'P09', b'P09', b'P09', '', '',
        b'P15', b'P15', '', b'P19', '', b'P26', '', ''],
       [b'P35', b'P35', b'P35', b'P35', b'P35', '', b'P37', b'P37', '',
        b'P39', '', '', '', '', '', '', '', b'P04', b'P04', b'P04', b'P04',
        b'P04', b'P04', b'P04', b'P04', b'P04', b'P04', b'P04', b'P04',
        b'P04'],
       [b'P72', b'P72', '', '', b'P73', b'P73', '', '', b'P75', '', '',
        b'P77', b'P77', b'P77', '', '', '', '', '', b'P60', '', '', b'P61',
        

In [699]:
month2 = month1.copy()
month3 = month1.copy()

In [722]:
manufacture_sequence['Product_ID'] = np.concatenate((month1.reshape(-1, 1), month1.reshape(-1, 1), month1.reshape(-1, 1)), axis=0)

In [710]:
manufacture_sequence.sort_values(by=['Month', 'Plant', 'Line', 'Day'], inplace = True)

In [746]:
manufacture_sequence['Product_ID'] = manufacture_sequence['Product_ID'].map(lambda x: str(x)[2:-1])

In [749]:
manufacture_sequence.to_csv('manufacture_sequence.csv', index=False)

In [798]:
product = 'P04'
for i in range(month1.shape[0]):
    if product in month1[i]:
        print(plant_line_inv[i][0])

B


In [800]:
def give_plant(product):
    for i in range(month1.shape[0]):
        if product in month1[i]:
            return (plant_line_inv[i][0])

In [813]:
def give_demand(product, region):
    return mforecast[(mforecast['Product_ID']==product)&(mforecast['Region']==region)]['Demand'].values[0]

In [817]:
sr = shipping_region.copy()

In [834]:
ship = [0 for i in range(sr.shape[0])]

In [835]:
for product,region in zip(sr['Product_ID'], sr['Region']):
    plant = give_plant(product)
    demand = give_demand(product, region)
    temp = sr[(sr['Product_ID']==product)&(sr['Region']==region)&(sr['Plant']==plant)]
    for idx in temp.index:
        ship[idx]=ceil(demand)

In [839]:
shipping_region[shipping_region.columns[-1]] = [*map(ceil, ship)]
shipping_region.to_csv('shipping_region.csv', index = False)

In [829]:
temp = []
for product in shipping_region['Product_ID']:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
sr['Product_ID'] = temp

temp = []
for product in shipping_region['Region']:
    if len(product)==2:
        temp.append(product[0]+'0'+product[1])
    else:
        temp.append(product)
sr['Region'] = temp

In [830]:
sr.head()

,Plant,Region,Product_ID,Month,Shipping_to_region_quantity
0,A,R01,P01,37,0
1,A,R01,P02,37,0
2,A,R01,P03,37,0
3,A,R01,P04,37,0
4,A,R01,P05,37,0


In [883]:
ms = pd.read_csv('manufacture_sequence.csv')
temp=[]
for i in ms['Product_ID']:
    if type(i) == str:
        if i[1]=='0':
            temp.append(i[0]+i[2])
        else:
            temp.append(i)
    else:
        temp.append(None)

In [885]:
ms['Product_ID'] = temp

In [888]:
ms.to_csv('manufacture_sequence.csv', index=False)

In [875]:
type(ms['Product_ID'][0])

str

In [892]:
#remove'P57' from a1
month1[0][0]=''
#moving 'P4' in b2 2days back
month1[3][14:-2] = 'P04'
month1[3][-2:] = ''
#moving 1 day forward in b3(only for first 14 days)
month1[4][1:3] = 'P72'
month1[4][5:7] = 'P73'
month1[4][9] = 'P75'
month1[4][12:15] = 'P77'
temp=[0, 4, 8, 11]
for i in temp:
    month1[4][i] = ''

In [893]:
month1

chararray([['', '', '', 'P58', 'P58', '', '', 'P64', '', '', 'P65', '', '',
        'P66', 'P66', '', '', 'P67', '', '', 'P68', 'P68', 'P68', '', '',
        'P69', '', '', 'P74', ''],
       ['P03', '', '', 'P13', '', 'P22', 'P22', 'P22', '', 'P33', '',
        'P34', '', 'P36', '', '', '', 'P43', 'P43', 'P43', '', 'P47', '',
        'P51', 'P51', '', '', '', '', ''],
       ['P23', 'P23', 'P23', 'P23', 'P23', '', '', '', '', '', '', '', '',
        'P08', 'P08', '', 'P09', 'P09', 'P09', 'P09', '', '', 'P15', 'P15',
        '', 'P19', '', 'P26', '', ''],
       ['P35', 'P35', 'P35', 'P35', 'P35', '', 'P37', 'P37', '', 'P39', '',
        '', '', '', 'P04', 'P04', 'P04', 'P04', 'P04', 'P04', 'P04', 'P04',
        'P04', 'P04', 'P04', 'P04', 'P04', 'P04', '', ''],
       ['', 'P72', 'P72', '', '', 'P73', 'P73', '', '', 'P75', '', '',
        'P77', 'P77', 'P77', '', '', '', '', 'P60', '', '', 'P61', '', '',
        'P62', '', '', 'P71', ''],
       ['P14', '', 'P16', '', 'P17', '', 'P24'

In [894]:
month2 = month1.copy()
month3 = month1.copy()

In [900]:
manufacture_sequence['Product_ID'] = np.concatenate((month1.reshape(-1, 1), month1.reshape(-1, 1), month1.reshape(-1, 1)), axis=0)

In [906]:
manufacture_sequence.sort_values(by=['Plant', 'Line', 'Month', 'Day'], inplace = True)

In [907]:
manufacture_sequence['Product_ID'] = manufacture_sequence['Product_ID']

In [908]:
manufacture_sequence.to_csv('manufacture_sequence.csv', index=False)

In [904]:
manufacture_sequence

,Plant,Line,Month,Day,Product_ID
0,A,1,37,1,
1,A,1,37,2,
2,A,1,37,3,
3,A,1,37,4,P58
4,A,1,37,5,P58
5,A,1,37,6,
6,A,1,37,7,
7,A,1,37,8,P64
8,A,1,37,9,
9,A,1,37,10,


In [909]:
#removed P61,P62

In [911]:
sq = pd.read_csv('shipping_region.csv')
sq[sq.columns[-1]] = sq[sq.columns[-1]]/2
sq.to_csv('shipping_region.csv', index=False)

In [915]:
sq[sq['Product_ID']=='P13']['Shipping_to_region_quantity'].sum()

420.0

In [917]:
give_plant('P73')

'B'

In [ ]:
reg_demand = forecast[(forecast['Product_ID']==product)&(forecast['Month']==month)]['Demand']
reg_demand_ratio = reg_demand/reg_demand.sum()
ship_quant = list(reg_demand_ratio*shipped)

In [3]:
demand

NameError: name 'demand' is not defined